In [3]:
import pandas as pd
import ast

In [4]:
df_movies = pd.read_csv("tmdb_5000_movies.csv")

In [5]:
df_movies.head(5)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


What I'm doing here is extracting only the useful columns,like the overview, keywords, and the companies that produced that movie, just to make our search recommendations just a little more extensive. 
I'm combining those three features into one, before I start processing them into word embeddings.

In [6]:
def extract_names(_str):
    try:
        # Convert the string to a list of dictionaries (This isnt necessary. I am only doing this because of the nature of our dataset)
        _list = ast.literal_eval(_str)
        # Extract the "name" attribute from each dictionary in the list
        _names = [_dict["name"] for _dict in _list]
        return ', '.join(_names).lower()
    except (ValueError, TypeError):
        # Handle potential errors or missing data, very important because not all rows have genres listed
        return []




In [7]:
# Apply the function using lambda to the "genres" column
df_movies["genres"] = df_movies["genres"].apply(lambda x: extract_names(x))
df_movies["keywords"] = df_movies["keywords"].apply(lambda x: extract_names(x))
df_movies["production_companies"] = df_movies["production_companies"].apply(lambda x: extract_names(x))

df_movies["tags"] =  df_movies["overview"]+" "+ df_movies["genres"] + " " + df_movies["keywords"] +" "+df_movies["production_companies"]
df_movies["tags"][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action, adventure, fantasy, science fiction culture clash, future, space war, space colony, society, space travel, futuristic, romance, space, alien, tribe, alien planet, cgi, marine, soldier, battle, love affair, anti war, power relations, mind and soul, 3d ingenious film partners, twentieth century fox film corporation, dune entertainment, lightstorm entertainment'

In [8]:
new_df = df_movies[["id", "original_title", "tags"]]

In [9]:
new_df['tags'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action, adventure, fantasy, science fiction culture clash, future, space war, space colony, society, space travel, futuristic, romance, space, alien, tribe, alien planet, cgi, marine, soldier, battle, love affair, anti war, power relations, mind and soul, 3d ingenious film partners, twentieth century fox film corporation, dune entertainment, lightstorm entertainment'

In [10]:
new_df.head()

,id,original_title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [11]:
new_df = new_df.dropna()

In [12]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4800 entries, 0 to 4802
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              4800 non-null   int64 
 1   original_title  4800 non-null   object
 2   tags            4800 non-null   object
dtypes: int64(1), object(2)
memory usage: 150.0+ KB


Now its time to preprocess the tags column and turn it to word embeddings!

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words = "english", max_features = 5000)

In [15]:
embeddings = tfidf.fit_transform(new_df["tags"]).toarray()
embeddings

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [16]:
len(tfidf.get_feature_names_out()) #Quickly checking how long my embedding list is

5000

In [17]:
from sklearn.metrics.pairwise import cosine_similarity
similarities = cosine_similarity(embeddings) #Compute similarites between each embedding vector and every other vector (incl. itself)

Function to handle recommendation

In [18]:
def recommend(movie_name):
    try:
        idx = new_df[new_df["original_title"] == movie_name].index[0]
        distances = similarities[idx]
        recommendations = sorted(list(enumerate(distances)), reverse = True, key=lambda x:x[1])[0:10]

        for i in recommendations:
            print(new_df.iloc[i[0]].original_title)
    except(IndexError):
        return "Movie does not exist in Catalogue"

In [26]:
recommend("Cars 2")

Cars 2
Cars
Herbie Fully Loaded
Return to Never Land
Monsters University
The Final Destination
Furious 7
The Fast and the Furious: Tokyo Drift
The Good Dinosaur
The Jungle Book 2


In [20]:
import pickle
pickle.dump(new_df, open("movie_list.pkl", 'wb'))
pickle.dump(similarities, open("similarities.pkl", 'wb'))